In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import tensorflow_datasets as tfds

In [ ]:
(train, valid, test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:70%]', 'train[70%:90%]', 'train[90%:]'],
    shuffle_files=True,
    as_supervised=True,  # Returns tuple (img, label)
    with_info=True,      # Also return the dataset info
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.VJXNXC_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [ ]:
def dataPreProcessing(image,label):
  image = tf.image.resize(image,[128,128])
  image = image/ 255
  return image,label

In [ ]:
train = train.map(dataPreProcessing)
valid = valid.map(dataPreProcessing)
test = test.map(dataPreProcessing)

In [ ]:
test = test.batch(32)
train = train.batch(32)
valid = valid.batch(32)

In [ ]:
random.seed(0)
model = models.Sequential()

model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.30))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.30))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.30))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.15))
model.add(layers.Dense(1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
lr = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train, validation_data=valid, epochs=40, verbose=1)

Epoch 1/40
509/509 ━━━━━━━━━━━━━━━━━━━━ 51s 79ms/step - accuracy: 0.5105 - loss: 0.6915 - val_accuracy: 0.5691 - val_loss: 0.6558
Epoch 2/40
509/509 ━━━━━━━━━━━━━━━━━━━━ 70s 71ms/step - accuracy: 0.6083 - loss: 0.6339 - val_accuracy: 0.6407 - val_loss: 0.5746
Epoch 3/40
509/509 ━━━━━━━━━━━━━━━━━━━━ 31s 61ms/step - accuracy: 0.6791 - loss: 0.5647 - val_accuracy: 0.6682 - val_loss: 0.5403
Epoch 4/40
509/509 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.7084 - loss: 0.5339 - val_accuracy: 0.6955 - val_loss: 0.5166
Epoch 5/40
509/509 ━━━━━━━━━━━━━━━━━━━━ 36s 71ms/step - accuracy: 0.7372 - loss: 0.4983 - val_accuracy: 0.7438 - val_loss: 0.4844
Epoch 6/40
509/509 ━━━━━━━━━━━━━━━━━━━━ 31s 61ms/step - accuracy: 0.7595 - loss: 0.4698 - val_accuracy: 0.7322 - val_loss: 0.4713
Epoch 7/40
509/509 ━━━━━━━━━━━━━━━━━━━━ 43s 64ms/step - accuracy: 0.7719 - loss: 0.4491 - val_accuracy: 0.7455 - val_loss: 0.4568
Epoch 8/40
509/509 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.7863 - loss: 0.4293 - 

In [ ]:
test_loss, test_accuracy = model.evaluate(test)
print(f"Test accuracy: {test_accuracy}")
print(f"Test loss: {test_loss}")

73/73 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.8454 - loss: 0.4213
Test accuracy: 0.8564058542251587
Test loss: 0.40015771985054016


In [ ]:
for image, label in test.take(1):
  plt.figure(figsize=(10,10))
  for i in range(3):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image[i])
    plt.title(int(labels[i]))
    plt.axis("off")